#Install Library

Install library yang dibutuhkan

In [1]:
!pip install hpelm scikit-learn pandas openpyxl matplotlib
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 654.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.6 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23

Import library yang akan digunakan

In [103]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from hpelm import ELM
import joblib

#Load & Preprocessing Data

Load data yang akan diproses

In [104]:
file_path = "/content/Data big.xlsx"  # Tentukan path file Excel yang akan dibaca
df = pd.read_excel(file_path, header=3).dropna()  # Baca file Excel mulai dari baris ke-4 (index 3), lalu hapus baris yang mengandung nilai kosong (NaN)

Ubah nama kolom agar lebih mudah diingat (opsional)

In [105]:
df.columns = [
    "No","Umur","Jenis Kelamin","Nyeri Dada","Tekanan Darah","Kolesterol",
    "Gula Darah","Hasil EKG","Detak Jantung", "Sakit Dada Selama Beraktivitas",
    "Atribut Hasil Prediksi Penyakit Jantung", "Hasil"]

Cek kolom untuk memastikan data yang akan diproses

In [106]:
print(df)

      No  Umur  Jenis Kelamin  Nyeri Dada  Tekanan Darah  Kolesterol  \
0      1    37              2           3            130         250   
1      2    41              1           2            130         204   
2      3    56              2           2            120         236   
3      4    57              1           4            120         354   
4      5    57              2           4            140         192   
..   ...   ...            ...         ...            ...         ...   
438  439    59              2           1            117         185   
439  440    51              2           3            179         220   
440  441    55              1           1            118         180   
441  442    16              1           1            112         180   
442  443    60              2           2            138         215   

     Gula Darah  Hasil EKG  Detak Jantung  Sakit Dada Selama Beraktivitas  \
0             0          0            187                 

Hapus kolom 'No' karena bukan termasuk data prediktif

In [107]:
df = df.drop(columns=["No"])

Ubah data 'Jenis Kelamin' kedalam bentuk biner

In [109]:
df['Jenis Kelamin'] = df['Jenis Kelamin'].replace({1: 0, 2: 1})

Menentukan target untuk memprediksi

In [111]:
X = df.drop(columns=["Hasil"])
y = df["Hasil"]

Preprocessing data jika target bukan numerik, encode ke angka

In [112]:
if y.dtype == object:
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    joblib.dump(label_encoder, "label_encoder.pkl")  # Optional: Simpan label encoder jika dibutuhkan nanti

Membagi data training 70% dan data testing 30%

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Normalisasi data agar lebih optimal

In [114]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Mengubah label target menjadi bentuk numerik biner (one-hot) agar bisa digunakan oleh model ELM untuk klasifikasi.

In [115]:
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

#Implementasi model ELM

Traning model ELM

In [116]:
elm = ELM(X_train_scaled.shape[1], y_train_encoded.shape[1], classification="c")
elm.add_neurons(150, "sigm")  # Jumlah neuron tersembunyi & fungsi aktivasi
elm.train(X_train_scaled, y_train_encoded)

Prediksi & evaluasi model ELM

In [117]:
y_pred = elm.predict(X_test_scaled).argmax(axis=1)

print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 0.631578947368421

Laporan klasifikasi:
              precision    recall  f1-score   support

           0       0.76      0.72      0.74        47
           1       0.74      0.72      0.73        47
           2       0.55      0.38      0.44        16
           3       0.36      0.53      0.43        15
           4       0.22      0.25      0.24         8

    accuracy                           0.63       133
   macro avg       0.53      0.52      0.52       133
weighted avg       0.65      0.63      0.64       133



Save model

In [84]:
elm.save("elm_model.h5") # Save model using elm.save()
joblib.dump(scaler, "scaler.pkl")
joblib.dump(encoder, "encoder.pkl")

['encoder.pkl']